In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import pandas as pd
# import ikpy.chain
# import ikpy.utils.plot as plot_utils
import itertools
import csv
import torch

import kinematics

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

joint_configs = np.loadtxt("joints_data_10_1024_test.csv", delimiter=',', skiprows=1)
joint_configs_tensor = torch.tensor(joint_configs[:, 1:5], dtype=torch.float64).to(device)

N_USERS = joint_configs_tensor.view(-1, 1024, 4).shape[0]  # (N_USERS, M_POSES, 4)
joint_configs_tensor.shape

In [ ]:
task_points = kinematics.forward_kinematics_pytorch(joint_configs_tensor)

indices = torch.arange(N_USERS)
indices = torch.repeat_interleave(indices, repeats=joint_configs_tensor.shape[0] // N_USERS).to(device)
task_points = torch.hstack((indices.unsqueeze(1), task_points))
print(task_points.shape)
print(task_points[0])
print(task_points[-1])

In [ ]:
csv_filename = "task_sp_points_10_1024_test.csv"
with open(csv_filename, 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["index", "x", "y", "z"])
    for row in task_points:
        csv_writer.writerow([int(row[0].cpu().numpy()), *row[1:].cpu().numpy()])